In [1]:
import torch
from transformers import RobertaModel, RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

c:\Users\Ridvan\anaconda3\envs\cudaPytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Define model names
roberta_model_name = 'roberta-base-peft'  # PEFT stands for Pretrained Error-Fix Transformer
modified_base = 'roberta-base-modified'  # A modified version of the RoBERTa base model
base_model = 'roberta-base'  # The original RoBERTa base model

In [3]:
# Load dataset
dataset = load_dataset('ag_news')  # Load the AG News dataset

In [4]:
# Initialize tokenizer using the base RoBERTa model
tokenizer = RobertaTokenizer.from_pretrained(base_model)

In [5]:
# Define a preprocessing function
def preprocess(examples):
    # Tokenize the text data, truncating and padding sequences
    tokenized = tokenizer(examples['text'], truncation=True, padding=True)
    return tokenized

In [6]:
# Apply preprocessing to the entire dataset in batches, removing the 'text' column
tokenized_dataset = dataset.map(preprocess, batched=True, remove_columns=["text"])

In [7]:
# Split the tokenized dataset into train, evaluation, and test datasets
train_dataset = tokenized_dataset['train']  # Training dataset
eval_dataset = tokenized_dataset['test'].shard(num_shards=2, index=0)  # Evaluation dataset (1st shard)
test_dataset = tokenized_dataset['test'].shard(num_shards=2, index=1)  # Test dataset (2nd shard)

In [8]:
# Extract the number of classess and their names
num_labels = dataset['train'].features['label'].num_classes
class_names = dataset["train"].features["label"].names
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

number of labels: 4
the labels: ['World', 'Sports', 'Business', 'Sci/Tech']


In [9]:
# Create an id2label mapping
# We will need this for our classifier.
id2label = {i: label for i, label in enumerate(class_names)}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

In [17]:
# use the same Training args for all models

# Define training arguments for model training
training_args = TrainingArguments(
    output_dir='./results',  # Directory to save the model and training results
    evaluation_strategy='steps',  # Evaluate on the validation set every 'steps' training steps
    learning_rate=5e-5,  # Initial learning rate for the optimizer
    num_train_epochs=1,  # Number of times to iterate over the entire training dataset
    per_device_train_batch_size=16,  # Batch size for each GPU during training
)

In [18]:
# Define a function to create a Trainer object for model training
def get_trainer(model):
    # Create a Trainer object with the specified arguments
    return Trainer(
        model=model,  # The model to be trained
        args=training_args,  # Training arguments for configuring the training process
        train_dataset=train_dataset,  # Training dataset
        eval_dataset=eval_dataset,  # Evaluation dataset
        data_collator=data_collator,  # Data collator for batch processing
    )

In [ ]:
# Create a Trainer object for full fine-tuning of the model
full_finetuning_trainer = get_trainer(
    AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label),
)
# Start training the model using the Trainer object
full_finetuning_trainer.train()

In [ ]:
#PEFT Training
# Creating a model for sequence classification based on a pretrained model
model = AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label)

# Configuring Lora parameters for PEFT (Proxy Experience Finetuning)
peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)

# Applying Lora parameters to the model to create the PEFT model
peft_model = get_peft_model(model, peft_config)

# Printing information about the PEFT model's trainable parameters
print('PEFT Model')
peft_model.print_trainable_parameters()

# Obtaining a trainer object for Lora finetuning
peft_lora_finetuning_trainer = get_trainer(peft_model)

# Initiating the training process for the PEFT model using Lora finetuning
peft_lora_finetuning_trainer.train()

In [ ]:
# Save
tokenizer.save_pretrained(modified_base)
peft_model.save_pretrained(peft_model_name)

In [ ]:
from peft import AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer


# LOAD the Saved PEFT model

inference_model = AutoPeftModelForSequenceClassification.from_pretrained(peft_model_name, id2label=id2label)
tokenizer = AutoTokenizer.from_pretrained(modified_base)

def classify(text):
    # Tokenize the input text, truncate if needed, pad sequences, and convert to PyTorch tensors
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    # Pass the tokenized input through the inference model to get predictions
    output = inference_model(**inputs)

    # Extract the predicted class index from the model output
    prediction = output.logits.argmax(dim=-1).item()

print(f'\n Class: {prediction}, Label: {id2label[prediction]}, Text: {text}')
  # return id2label[prediction]

In [ ]:
# Import necessary modules
from torch.utils.data import DataLoader  # DataLoader for creating data iterators
import evaluate  # Import module for computing evaluation metrics
from tqdm import tqdm  # tqdm for progress tracking during evaluation

# Load the evaluation metric (e.g., accuracy) from the evaluate module
metric = evaluate.load('accuracy')

# Define a function to evaluate the model on a given dataset
def evaluate_model(inference_model, dataset):
    # Create a DataLoader for the evaluation dataset with batch size 8 and using data_collator for batch processing
    eval_dataloader = DataLoader(dataset.rename_column("label", "labels"), batch_size=8, collate_fn=data_collator)
    # Determine the device (GPU or CPU) for model inference
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move the inference model to the appropriate device (GPU or CPU)
    inference_model.to(device)
    # Set the model in evaluation mode (no gradient computation)
    inference_model.eval()

    # Iterate over batches in the evaluation dataloader
    for step, batch in enumerate(tqdm(eval_dataloader)):
        # Move the batch to the device (GPU or CPU)
        batch.to(device)
        # Disable gradient calculation for inference
        with torch.no_grad():
            # Perform inference using the inference model on the batch
            outputs = inference_model(**batch)
        # Extract the predicted class indices from the model outputs
        predictions = outputs.logits.argmax(dim=-1)
        # Assign predictions and reference labels to variables for metric computation
        predictions, references = predictions, batch["labels"]
        # Add the batch of predictions and references to the evaluation metric
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    # Compute the evaluation metric (e.g., accuracy) based on added batches
    eval_metric = metric.compute()
    # Print the computed evaluation metric
    print(eval_metric)

In [ ]:
# Evaluate the non fine-tuned model
evaluate_model(AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label), test_dataset)

In [ ]:

# Evaluate the PEFT fine-tuned model
evaluate_model(inference_model, test_dataset)

In [ ]:
# Evaluate the Fully fine-tuned model
evaluate_model(full_finetuning_trainer.model, test_dataset)